# Imports

In [ ]:
!pip install --upgrade pyalgotrading

In [1]:
from pyalgotrading.algobulls import AlgoBullsConnection

# Connection

In [2]:
connection = AlgoBullsConnection()

In [3]:
connection.get_token_url()

Please login to this URL to get your unique token: https://app.algobulls.com/settings?section=developerOptions


In [4]:
API_TOKEN = "71d4e9c405acc3f595ff62eaf96806127d921292"
connection.set_access_token(API_TOKEN)

Access token is valid.


# Strategy Creation

## Import Strategy from pyaglostrategypool

In [ ]:
! wget -O rsi_macd_crossover.py https://raw.githubusercontent.com/algobulls/pyalgostrategypool/master/pyalgostrategypool/rsi_macd_crossover/_strategy.py
! sed -i '1s/^/from pyalgotrading.strategy import StrategyBase\n/' rsi_macd_crossover.py

In [5]:
from rsi_macd_crossover import RSIMACDCrossover as strategy_cls

In [6]:
response = connection.create_strategy(strategy_cls, overwrite=True)
response

{'message': 'Strategy saved successfully',
 'cstcId': 135403,
 'strategyId': '551a08e21d104dd1907b21ffe1252281'}

In [7]:
strategy = response['strategyId']

# Strategy Testing

## Instruments Searching (optional)

In [8]:
instrument = connection.search_instrument('HEROMOTOCO', exchange='NSE')
instrument

[{'id': 28, 'value': 'NSE:HEROMOTOCO'},
 {'id': 1672355, 'value': 'NSE:HEROMOTOCO.AF'},
 {'id': 1672479, 'value': 'NSE:HEROMOTOCO.BL'},
 {'id': 2316395, 'value': 'NSE:HEROMOTOCO.L1'},
 {'id': 2041177, 'value': 'NSE:HEROMOTOCO.R1'},
 {'id': 2041176, 'value': 'NSE:HEROMOTOCO.V1'},
 {'id': 24609, 'value': 'NSE:HEROMOTOCO20NOV2150CE'},
 {'id': 24610, 'value': 'NSE:HEROMOTOCO20NOV2150PE'},
 {'id': 24611, 'value': 'NSE:HEROMOTOCO20NOV2200CE'},
 {'id': 24612, 'value': 'NSE:HEROMOTOCO20NOV2200PE'}]

## Setup Parameters

In [9]:
parameters = {
    'OVERBOUGHT_VALUE': 85,
    'OVERSOLD_VALUE': 15,
    'TIMEPERIOD_FAST': 12,
    'TIMEPERIOD_RSI': 3,
    'TIMEPERIOD_SIGNAL': 9,
    'TIMEPERIOD_SLOW': 26
}

In [10]:
initial_virtual_funds = 1000000000     # in Rupees

## Backtesting

### Start

In [11]:
connection.backtest(
    strategy=strategy, 
    start='2021-08-01 09:15 +0530', 
    end='2023-07-31 15:30 +0530', 
    instrument='NSE:HEROMOTOCO', 
    lots=2,
    parameters=parameters, 
    candle='1 hour',
    initial_funds_virtual=initial_virtual_funds

)


Starting the strategy 'RSI MACD Crossover V2' in BACKTESTING mode...
╒═════════════════════════╤═══════════════════════════╕
│ Config                  │ Value                     │
╞═════════════════════════╪═══════════════════════════╡
│ Strategy Name           │ RSI MACD Crossover V2     │
├─────────────────────────┼───────────────────────────┤
│ Trading Type            │ BACKTESTING               │
├─────────────────────────┼───────────────────────────┤
│ Instrument(s)           │ ['NSE:HEROMOTOCO']        │
├─────────────────────────┼───────────────────────────┤
│ Quantity/Lots           │ 2                         │
├─────────────────────────┼───────────────────────────┤
│ Start Timestamp         │ 2021-08-01 09:15:00+05:30 │
├─────────────────────────┼───────────────────────────┤
│ End Timestamp           │ 2023-07-31 15:30:00+05:30 │
├─────────────────────────┼───────────────────────────┤
│ Parameters              │ {'OVERBOUGHT_VALUE': 85,  │
│                         │  'OVER

### Status

In [12]:
connection.get_backtesting_job_status(strategy)

{'message': 'STARTING', 'status': 1}

### Logs

In [13]:
logs = connection.get_backtesting_logs(strategy)

Looking for a dedicated virtual server to execute your strategy... (10)

Execution Progress:   0%|          | 0/63008100.0 [00:00<?]

In [14]:
print(logs)

[2023-10-23 18:56:16] Performing sanity checks on cfg strategy_parameters, setting up required data structures...
[2023-10-23 18:56:16] Sanity checks on cfg successful.
[2023-10-23 18:56:16] Currency set to "INR"
[2023-10-23 18:56:16] Setting up broker connection...
[2023-10-23 18:56:17] Broker connection has been setup successfully.
[2023-10-23 18:56:17] (NSE_EQ) Funds available in client's ABVIRTUALBROKER account is : INR '1000000000.00'
[2023-10-23 18:56:17] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.3.0)... 
########################################
[2023-10-23 18:56:17] Welcome ALGOBULLS VIRTUAL USER!
[2023-10-23 18:56:17] Reading strategy...
[2023-10-23 18:56:18] Entering Backtesting mode. Henceforth, all timestamps will be Backtesting timestamps...
[BT] [2021-08-01 09:15:00,000] [INFO] [tls] STARTING ALGOBULLS CORE...
[BT] [2021-08-01 09:15:00,000] [INFO] [tls] 
  #####  #######    #    ######  ####### ### #     #  #####     
 #     #    #      # #

### Stop

In [15]:
connection.stop_backtesting_job(strategy)

Stopping BACKTESTING job... Success.


### Profit and Loss Reports

In [ ]:
pnl_reports = connection.get_backtesting_report_pnl_table(strategy)
pnl_reports

### Statistics Reports

#### Statistics

In [ ]:
connection.get_backtesting_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_backtesting_report_statistics(strategy, report='full', html_dump=True )

### Order History

In [ ]:
order_history = connection.get_backtesting_report_order_history(strategy)
print(order_history)

---

## Papertrading

### Start

In [ ]:
connection.papertrade(
    strategy=strategy, 
    start='9:15 +0530', 
    end='15:00 +0530',  
    instruments='NSE:ADANIPOWER', 
    lots=5,
    parameters=parameters, 
    candle='1 minute',
)

### Status

In [ ]:
connection.get_papertrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_papertrading_logs(strategy)
print(logs)

### Stop

### Profit and Loss Reports (Paper Trading)

In [ ]:
pnl_reports = connection.get_papertrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Paper Trading)

#### Statistics

In [ ]:
connection.get_papertrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_papertrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)

### Order History (Paper Trading)

In [ ]:
order_history = connection.get_papertrading_report_order_history(strategy)
print(order_history)

---